In [26]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict,Literal,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from dotenv import load_dotenv

In [27]:
load_dotenv()

True

In [28]:
class ChatbotState(TypedDict):
    messages: Annotated[list[BaseMessage],add_messages]

In [29]:
llm = ChatOpenAI(model="gpt-4o-mini")

def chat_node(state:ChatbotState):
    
    response = llm.invoke(state["messages"])
    return {"messages":[response]}


In [31]:
# add checkpointer to utilize local memory for chat history
checkpointer = MemorySaver()
# create graph with nodes and edges
graph = StateGraph(ChatbotState)

graph.add_node("chat_node",chat_node)

graph.add_edge(START,"chat_node")
graph.add_edge("chat_node",END)
# compile graph to create chatbot object
chatbot = graph.compile(checkpointer=checkpointer)

In [35]:
# create a thread id for chat history
thread_id = "1"
# start chatting with AI chatbot
while True:
    user_query = input("type here:")
    print(f"\nUser: {user_query}")
    
    # loop exit criteria
    if user_query.strip().lower() in ['exit','bye','end','quit']:
        break 

    #create config for memory allocation to chat history
    CONFIG = {'configurable':{'thread_id':thread_id}}
    #response = chatbot.invoke({"messages": [HumanMessage(content=user_query)]},config=config)
    print(f"AI: ")
    for msg_chunk,metadata in chatbot.stream({'messages':HumanMessage(content=user_query)},
                                             config=CONFIG,
                                             stream_mode="messages"):
        if msg_chunk.content:
            print(msg_chunk.content,end=" ",flush=True)

    



User: ho
AI: 
Hi !  It  seems  like  there  might  have  been  a  small  typo  there .  How  can  I  assist  you ? 
User: write a brief summary on AI
AI: 
Artificial  Intelligence  ( AI )  refers  to  the  simulation  of  human  intelligence  in  machines  designed  to  think  and  act  like  humans .  It  encompasses  various  technologies ,  including  machine  learning ,  natural  language  processing ,  robotics ,  and  computer  vision .  AI  systems  can  analyze  vast  amounts  of  data ,  recognize  patterns ,  make  decisions ,  and  automate  tasks ,  improving  efficiency  and  productivity  across  numerous  sectors ,  such  as  healthcare ,  finance ,  transportation ,  and  entertainment .

 The  development  of  AI  raises  important  ethical  considerations ,  including  issues  of  bias ,  privacy ,  and  the  potential  impact  on  jobs .  As  AI  technology  continues  to  advance ,  it  is  becoming  increasingly  integrated  into  everyday  life ,  driving  innova